In [4]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0.1)



examples = [
    {
        "input": "탑건",
        "output": """
            🛩️👨‍✈️🔥
        """
    }, 
    {
        "input": "대부",
        "output": """
            👨‍👨‍👦🔫🍝
        """
    }    
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
    memory_key="chat_history"
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 사용자의 입력과 일치하는 영화를 찾고, 그 영화를 세개의 이모티콘으로 표현합니다."),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | final_prompt | llm

def invoke_chain(input):
    result = chain.invoke({"input": input})
    memory.save_context(
        {"input":input},
        {"output": result.content}
    )
    print(result)

invoke_chain("라라랜드")
invoke_chain("앞에서 말한 영화제목은 뭐지?")

content='\n            🎶🌆❤️\n        '
content='탑건, 대부, 라라랜드였어요!'
